In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from surprise import Reader, Dataset, SVD
from sklearn.model_selection import KFold
from surprise.model_selection import cross_validate


In [21]:

reader = Reader()

ratings_small = pd.read_csv('../data/ratings_small.csv')
links_small = pd.read_csv('../data/links_small.csv')

ratings_small['movieId'] = ratings_small['movieId'].astype('int')
links_small['movieId'] = links_small['movieId'].astype('int')

links_small = links_small.merge(ratings_small,on='movieId')
links_smalls = links_small[links_small['tmdbId'].notnull()]['tmdbId'].astype('int')

links_small.head()

,movieId,imdbId,tmdbId,userId,rating,timestamp
0,1,114709,862.0,7,3.0,851866703
1,1,114709,862.0,9,4.0,938629179
2,1,114709,862.0,13,5.0,1331380058
3,1,114709,862.0,15,2.0,997938310
4,1,114709,862.0,19,3.0,855190091


linksmall이랑 ratings_small merge 해야됨

In [22]:
md = pd. read_csv('../data/movies_metadata.csv')
md.head()
md = md[['title','id']]
md.rename(columns = {'id' : 'tmdbId'},inplace=True)
md = md.drop([19730, 29503, 35587])
md['tmdbId'] = md['tmdbId'].astype('int')


links_small = links_small.merge(md,on='tmdbId')
links_small = links_small.sort_values(by=['userId'])
links_small.head()



/var/folders/dd/h1chfyx12gbg8nkt2z48_kr00000gn/T/ipykernel_67673/1839456158.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  md = pd. read_csv('../data/movies_metadata.csv')


,movieId,imdbId,tmdbId,userId,rating,timestamp,title
57496,2968,81633,36819.0,1,1.0,1260759200,Time Bandits
42609,1953,67116,1051.0,1,4.0,1260759191,The French Connection
36253,1405,115641,3179.0,1,1.0,1260759203,Beavis and Butt-Head Do America
25666,1061,117665,819.0,1,3.0,1260759182,Sleepers
34633,1343,101540,1598.0,1,2.0,1260759131,Cape Fear


In [23]:
data = Dataset.load_from_df(links_small[['userId', 'movieId', 'rating']], reader)
kf = KFold(n_splits=5)
kf.split(data)

<generator object _BaseKFold.split at 0x7f9381d17660>

In [24]:

svd = SVD(random_state=42)
cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8978  0.9007  0.8962  0.9026  0.8907  0.8976  0.0041  
MAE (testset)     0.6907  0.6949  0.6916  0.6945  0.6853  0.6914  0.0034  
Fit time          0.54    0.53    0.52    0.56    0.52    0.53    0.02    
Test time         0.09    0.10    0.09    0.09    0.20    0.11    0.04    


{'test_rmse': array([0.89779137, 0.90074201, 0.89622355, 0.90256255, 0.89071162]),
 'test_mae': array([0.69072639, 0.69493511, 0.69156632, 0.69446213, 0.68533638]),
 'fit_time': (0.5416750907897949,
  0.5347280502319336,
  0.5172982215881348,
  0.5554790496826172,
  0.5154378414154053),
 'test_time': (0.09112882614135742,
  0.09727311134338379,
  0.09004902839660645,
  0.0926210880279541,
  0.19821405410766602)}

In [25]:
trainset = data.build_full_trainset()
svd.fit(trainset)

# with open('recommendModel.pkl','wb') as f:
#     pickle.dump(svd,f)
# with open('recommendData.pkl','wb') as f:
#     pickle.dump(links_small,f)

In [26]:
links_small[links_small['userId'] == 1]

,movieId,imdbId,tmdbId,userId,rating,timestamp,title
57496,2968,81633,36819.0,1,1.0,1260759200,Time Bandits
42609,1953,67116,1051.0,1,4.0,1260759191,The French Connection
36253,1405,115641,3179.0,1,1.0,1260759203,Beavis and Butt-Head Do America
25666,1061,117665,819.0,1,3.0,1260759182,Sleepers
34633,1343,101540,1598.0,1,2.0,1260759131,Cape Fear
46595,2150,80801,8393.0,1,3.0,1260759194,The Gods Must Be Crazy
32372,1263,77416,11778.0,1,2.0,1260759151,The Deer Hunter
1611,31,112792,9909.0,1,2.5,1260759144,Dangerous Minds
35197,1371,79945,152.0,1,2.5,1260759135,Star Trek: The Motion Picture
27288,1129,82340,1103.0,1,2.0,1260759185,Escape from New York


In [27]:
rank = []
for columns,items in links_small.iterrows():
    test = []
    test.append(items[6])
    test.append(svd.predict(90, items[0])[3])
    rank.append(test)



In [28]:
print(rank[0])
print(rank[1])
print(rank[2])

df = pd.DataFrame(data = rank,columns= ['title','score'])
df.sort_values(ascending=False,by='score',inplace=True)
df.drop_duplicates(subset=None, keep='first', inplace=True, ignore_index=False)
df

['Time Bandits', 3.8296113601769664]
['The French Connection', 4.151184465651257]
['Beavis and Butt-Head Do America', 3.554384171641936]


,title,score
68095,The Godfather,4.624916
85651,It Happened One Night,4.621732
66471,Roger & Me,4.602859
5049,Raging Bull,4.596313
53885,The Shawshank Redemption,4.594331
...,...,...
74524,Mighty Morphin Power Rangers: The Movie,2.491238
51428,Wild Wild West,2.470930
74494,Spice World,2.468620
15070,Super Mario Bros.,2.373715
